# 09 Multi-Agent System (Experiment)
Read first: `examples/09_multi_agent_system.py`

Define specialized agents (architect/implementer/reviewer), tweak their instructions/tools, run a handoff workflow, and inspect outputs.


## Select a provider
Set `PROVIDER_KEY` to `sonnet`, `opus`, `gpt`, or `gpt-codex` and run the next cell.
The helper will set env vars (prompting if missing) and print the selection.


In [ ]:
# Choose your provider key (sonnet | opus | gpt | gpt-codex)
PROVIDER_KEY = "sonnet"

In [ ]:
# Resolve foundation/provider paths and ensure env var
import importlib.util
import os
from pathlib import Path


def find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for p in candidates:
        if (p / "utils" / "providers.py").exists():
            return p
    return Path.cwd()


ROOT = find_repo_root()
providers_path = ROOT / "utils" / "providers.py"
spec = importlib.util.spec_from_file_location("providers_helper", providers_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
select_provider = module.select_provider
print_provider_menu = module.print_provider_menu

FOUNDATION_PATH, PROVIDER_PATH, REQUIRED_ENV = select_provider(PROVIDER_KEY, foundation="minimal")
print_provider_menu(PROVIDER_KEY)
print(f"Selected: {PROVIDER_KEY} -> {PROVIDER_PATH.name}")
env_set = "Yes" if os.environ.get(REQUIRED_ENV) else "No"
print(f"Required env: {REQUIRED_ENV} (set? {env_set})")

In [ ]:
# Optional: set API key here
import getpass

if not os.getenv(REQUIRED_ENV):
    os.environ[REQUIRED_ENV] = getpass.getpass(f"Enter {REQUIRED_ENV}: ")
    print(f"Set {REQUIRED_ENV} for this kernel session.")
else:
    print(f"{REQUIRED_ENV} already set.")

In [ ]:
# Parameters (edit me)
TASK = "Create a Python module for rate limiting API requests"
PROMPTS = {
    "architect": "Design a rate limiting module (token bucket, sliding window). Include structure and interfaces.",
    "implementer": "Implement the rate limiter per the architect design. Provide code and a small test.",
    "reviewer": "Review the implementation for correctness, thread safety, and clarity. Provide actionable feedback.",
}

In [ ]:
from amplifier_foundation import Bundle
from amplifier_foundation import load_bundle


def create_architect(provider: Bundle) -> Bundle:
    return Bundle(
        name="architect",
        version="1.0.0",
        tools=[
            {
                "module": "tool-filesystem",
                "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            }
        ],
        instruction="You are a software architect. Design clear, scalable architectures.",
    ).compose(provider)


def create_implementer(provider: Bundle) -> Bundle:
    return Bundle(
        name="implementer",
        version="1.0.0",
        tools=[
            {
                "module": "tool-filesystem",
                "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            },
            {"module": "tool-bash", "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"},
        ],
        instruction="You are a software implementer. Write clean, tested code per the spec.",
    ).compose(provider)


def create_reviewer(provider: Bundle) -> Bundle:
    return Bundle(
        name="reviewer",
        version="1.0.0",
        tools=[
            {
                "module": "tool-filesystem",
                "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            }
        ],
        instruction="You are a code reviewer. Check correctness, safety, and clarity.",
    ).compose(provider)

In [ ]:
def format_context(task: str, previous: dict[str, str]) -> str:
    if not previous:
        return f"Task: {task}"
    parts = [f"Task: {task}", "Previous work:"]
    for name, output in previous.items():
        snippet = output[:400] + ("..." if len(output) > 400 else "")
        parts.append(f"\n{name.upper()} OUTPUT:\n{snippet}")
    return "\n".join(parts)


async def run_workflow():
    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(PROVIDER_PATH))
    architect = create_architect(provider)
    implementer = create_implementer(provider)
    reviewer = create_reviewer(provider)

    agents = [
        ("architect", PROMPTS["architect"], architect),
        ("implementer", PROMPTS["implementer"], implementer),
        ("reviewer", PROMPTS["reviewer"], reviewer),
    ]

    results = {}
    for name, instruction, agent_bundle in agents:
        composed = foundation.compose(agent_bundle)
        prepared = await composed.prepare()
        session = await prepared.create_session()
        context = format_context(TASK, results)
        full_prompt = f"{context}\n\n{instruction}"
        print(f"\nRunning {name}...")
        async with session:
            resp = await session.execute(full_prompt)
        results[name] = resp
        print(f"{name} output length: {len(resp)}")
    return results


results = await run_workflow()
print("\nSummary:")
for name, output in results.items():
    preview = output[:200] + ("..." if len(output) > 200 else "")
    print(f"- {name}: {len(output)} chars; preview: {preview}")

### Try this
- Change `TASK` and the role prompts in `PROMPTS` to your own domain.
- Add/remove tools in `create_*` functions to see how capabilities affect outputs.
- Insert another agent in the `agents` list (e.g., tester) and rerun.
- Swap `PROVIDER_PATH` to compare models in a multi-agent handoff.
